In [35]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [34]:
def timeToMinutes(time):
    splitted = time.split(":")
    return int(splitted[0]) * 60 + int(splitted[1])

In [36]:
def processTime(time):
    minutes = timeToMinutes(time)
    return  math.sin(minutes / (24 * 60)*math.pi)

In [43]:
def loadData():
    data = []
    for i in range(1, 70):
        filePrefix = 'Diabetes-Data/data-'
        if i < 10:
            filePrefix += '0'
        fileHandle = open(filePrefix+str(i), 'r')
        print(i)
        for line in fileHandle:
            line = line.strip()
            if line:
                try:
                    parts = line.split('\t')
                    time = processTime(parts[1])
                    if time >1 or time < 0:
                        print(time)
                        raise Exception('Time out of bounds')
                    code = int(parts[2])
                    number = int(parts[3])
                    data.append([time, code, number])
                except ValueError:
                    pass
                except IndexError:
                    pass
        
        fileHandle.close()
    return data


In [44]:
data = loadData()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67


Exception: Time out of bounds

In [15]:
def initializeParameters(nX, nH, nY):
    W1 = np.random.randn(nH, nX) * 0.01
    b1 = np.zeros((nH, 1))
    W2 = np.random.randn(nY, nH) * 0.01
    b2 = np.zeros((nY, 1))
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    return parameters

In [2]:
def costFunction(Y, Y_hat):
    n = Y.shape[0]
    return -(1/n) * np.sum(Y * np.log(Y_hat))


In [3]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

In [4]:
def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z), axis=0)

In [8]:
def propagate(X, Y, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    cost = costFunction(Y, A2)
    # pochodne funkcji straty:
    dA2 = A2 - Y
    dW2 = np.dot(A1.T,dA2) 
    db2 = np.sum(dA2, axis=0)
    dA1 = np.dot(dA2, W2.T)
    dZ1 = dA1 * (A1 * (1 - A1))
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0)
    gradients = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return gradients, cost

In [9]:
def updateParameters(W1, b1, W2, b2, gradients, learningRate):
    W1 = W1 - learningRate * gradients["dW1"]
    b1 = b1 - learningRate * gradients["db1"]
    W2 = W2 - learningRate * gradients["dW2"]
    b2 = b2 - learningRate * gradients["db2"]
    return W1, b1, W2, b2

In [12]:
def optimize(X, Y, W1, b1, W2, b2, learningRate, numberOfIterations, printCost=False):
    costs = []
    for i in range(numberOfIterations):
        gradients, cost = propagate(X, Y, W1, b1, W2, b2)
        W1, b1, W2, b2 = updateParameters(W1, b1, W2, b2, gradients, learningRate)
        costs.append(cost)
        if i % 100 == 0 and printCost:
            print("Cost after iteration {}: {}".format(i, cost))
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    return parameters, costs

$$
\sum_{i=1}^{\infty} \frac{1}{i} =\infty
$$